In [1]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Modelo simple

\begin{align}
\text{minimize} \qquad & x+y \\
 \text{subject to} \quad \quad & x+y \geq 1 \\
 \qquad \qquad & x \geq 0, y \geq 0 \\
 \qquad \qquad & x,y \in \mathbb{R}
\end{align}



In [26]:
#m = Model(solver = GLPKSolverMIP())
#m = Model(solver = GLPKSolverLP())

#creamos un modelo
m = Model(solver = CplexSolver())

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Min x + y
Subject to
 x + y ≥ 1
 x ≥ 0
 y ≥ 0
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Status: Optimal Objective value: 1.0
x = 0.0
y = 1.0
CPLEX Error  3003: Not a mixed-integer problem.


# Un programa lineal en formato estándar

Un PL en formato estándar se expresa como:

$$
\begin{align}
& \text{min} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


In [26]:
#Datos
#----------

#INPUT DATA
#----------

c=[2500; 4000; 0; 0; 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]
m, n = size(A) # m = filas n = columnas

(3,5)

In [28]:
modelo = Model(solver = CplexSolver())
@variable(modelo, x[1:n] >= 0) # Models x >=0
for i in 1:m # filas
    @constraint(modelo, sum(A[i, j]*x[j] for j in 1:n) == b[i]) # i-ésima restricción 
end 
@objective(modelo,Min,sum(c[i]*x[i] for i in 1:n))
status = solve(modelo) # solves the model  

Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 5 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
CPLEX Error  3003: Not a mixed-integer problem.


:Optimal

# Un modelo algo más complejo

Obviamente, el modelo anterior no tiene mayor dificultad. Alternativamente podríamos intentar resolver algún modelo con algo más de historia (como por ejemplo el Uncapacitated Lot Sizing Problem).

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_K y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+ \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}

$c_s$ y $c_K$ son los costes de inventario (unidad por periodo) y de realizar un pedido en un periodo y $d_t$ la demanda de cada periodo. El conjunto de periodos es $T$. $x_t$ es la cantidad que se recibe en el periodo $t$, mientras que $y_t$ equivale a si se realiza un pedido en el periodo $t$ o no y $s_t$ es el inventario al final del periodo $t$.

El objetivo minimiza la suma de costos de inventario y pedido (el costo por unidad no se tiene en cuenta porque la solución óptima siempre compra exactamente la demanda, $\sum_{t\in T} d_t$.

La primera restricción asegura el cumplimiento de la demanda de cada periodo. La segunda restricción asegura que sólo se compran unidades si se ha realizado un pedido ($M$ es un valor lo suficientemente grande, en este caso basta con $\sum_{t\in T} d_t$). La tercera restricción inicializa el inventario y el resto de restricciones definen el problema.

In [13]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cK=50
cs=2
M=sum(d)

108

In [24]:
modelo=Model(solver = CplexSolver())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cK*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

Min 2 s[1] + 2 s[2] + 2 s[3] + 2 s[4] + 2 s[5] + 2 s[6] + 2 s[7] + 2 s[8] + 2 s[9] + 2 s[10] + 50 y[1] + 50 y[2] + 50 y[3] + 50 y[4] + 50 y[5] + 50 y[6] + 50 y[7] + 50 y[8] + 50 y[9] + 50 y[10]
Subject to
 s[0] + x[1] - s[1] = 5
 s[1] + x[2] - s[2] = 10
 s[2] + x[3] - s[3] = 7
 s[3] + x[4] - s[4] = 11
 s[4] + x[5] - s[5] = 13
 s[5] + x[6] - s[6] = 14
 s[6] + x[7] - s[7] = 2
 s[7] + x[8] - s[8] = 8
 s[8] + x[9] - s[9] = 17
 s[9] + x[10] - s[10] = 21
 x[1] - 108 y[1] ≤ 0
 x[2] - 108 y[2] ≤ 0
 x[3] - 108 y[3] ≤ 0
 x[4] - 108 y[4] ≤ 0
 x[5] - 108 y[5] ≤ 0
 x[6] - 108 y[6] ≤ 0
 x[7] - 108 y[7] ≤ 0
 x[8] - 108 y[8] ≤ 0
 x[9] - 108 y[9] ≤ 0
 x[10] - 108 y[10] ≤ 0
 s[0] = 0
 x[i] ≥ 0 ∀ i ∈ {1,2,…,9,10}
 y[i] ∈ {0,1} ∀ i ∈ {1,2,…,9,10}
 s[i] ≥ 0 ∀ i ∈ {0,1,…,9,10}
Found incumbent of value 378.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 3 columns.
Reduced MIP has 18 rows, 28 columns, and 45 nonzeros.
Reduced MIP has 9 binaries, 0 generals, 0 SO

# Set Covering Problem

Veamos un ejemplo un poco más complejo en que es importante considerar que la mayoría de la matriz A está formada por ceros.

El modelo es:

\begin{align}
\text{minimize} \qquad & \sum_{j=1}^{n} c_j x_j \\
 \text{subject to} \quad \quad & \sum_{i=1}^{m} a_{ij} x_j \geq 1 \forall i\in 1,\cdots,m \\
 \qquad \qquad & x_j \in \{0,1\} \forall j\in 1,\cdots,n
\end{align}

Primero vamos a bajar un archivo de datos del mismo 

In [2]:
run(`wget -O scp41.txt http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt`)

--2017-09-26 16:33:08--  http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt
Resolving people.brunel.ac.uk (people.brunel.ac.uk)... 134.83.117.116
Connecting to people.brunel.ac.uk (people.brunel.ac.uk)|134.83.117.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20562 (20K) [text/plain]
Saving to: ‘scp41.txt’

     0K .......... ..........                                 100% 61.3K=0.3s

2017-09-26 16:33:10 (61.3 KB/s) - ‘scp41.txt’ saved [20562/20562]



In [3]:
run(`cat scp41.txt`)

 200 1000 
 1 1 1 1 1 1 1 1 1 1 1 1 
 2 2 2 2 2 2 2 2 2 2 2 2 
 2 2 2 3 3 3 3 3 3 3 3 3 
 3 3 3 3 3 3 4 4 4 4 4 4 
 4 4 4 4 4 4 4 4 5 5 5 5 
 5 5 5 5 5 6 6 6 6 6 6 6 
 6 6 6 6 7 7 7 7 7 7 7 7 
 7 7 7 8 8 8 8 8 8 8 8 8 
 8 9 9 9 9 9 9 9 9 10 10 10 
 10 10 10 10 10 10 11 11 11 11 11 11 
 11 12 12 12 12 12 12 12 12 12 12 12 
 12 12 12 13 13 13 13 13 13 13 13 14 
 14 14 14 14 14 14 14 14 14 14 14 14 
 14 15 15 15 15 15 15 15 15 15 15 15 
 15 15 15 15 15 16 16 16 16 16 16 17 
 17 17 17 17 18 18 18 18 18 18 18 18 
 18 18 18 18 18 19 19 19 19 19 19 19 
 20 20 20 20 20 20 20 20 20 20 20 20 
 20 21 21 21 21 21 21 21 21 22 22 22 
 22 22 22 22 22 22 22 23 23 23 23 23 
 23 23 23 23 23 23 24 24 24 24 24 24 
 24 24 24 24 24 25 25 25 25 25 25 25 
 25 26 26 26 26 26 26 26 26 27 27 27 
 27 27 27 27 27 27 27 27 27 27 28 28 
 28 28 28 28 28 29 29 29 29 29 29 29 
 29 30 30 30 30 30 31 31 31 31 31 31 
 31 31 31 31 32 32 32 32 32 32 32 32 
 32 32 32 33 33 33 33 33 33 33 33 33 
 33 34 34 34 34 34 34 34 35 35

La estructura de datos es algo confusa. Primero tenemos una línea con el número de restricciones (200) y en número de variables (1000). Los siguientes 1000 valores corresponden a los costos de las variables en la función objetivo y después hay la descripción de las 200 restricciones con esta forma (copio la primera)

 17 
 91 214 230 289 351 416 488 491 518 567 720 721 
 735 753 768 928 990 
 
 Que significa que hay 17 no ceros y que las variables 91, ..., 990 adoptan valor en la restriccion.
 
 Como la estructura es un tanto oscura, vamos a tener que crear nuestro propio lector de instancias:

In [51]:
function readFile(filename)
    f=open(filename,"r")
    s=readstring(f)
    close(f)
    s=replace(s,"\n","")
    s=split(s," ")
    nC=parse(Int64,s[2])
    nV=parse(Int64,s[3])
    numCoefFo=0
    numCoef=0
    numRow=0
    c= Float64[]
    I=Int64[]
    J=Int64[]
    V=Float64[]
    for i in 4:length(s)
        if s[i]!= ""
            if numCoefFo < nV
                push!(c,parse(Float64,s[i]))
                numCoefFo += 1
            else
                if numCoef == 0
                    numRow += 1
                    numCoef = parse(Int64,s[i])
                else
                    numCoef -= 1
                    push!(I,numRow)
                    push!(J,parse(Int64,s[i]))
                    push!(V,1.0)
                end
            end
        end
    end
    A=sparse(I,J,V)
    return nC,nV,c,A #,c,A
end

readFile (generic function with 1 method)

In [53]:
constraints,variables,c,A = readFile("scp41.txt")

println("\n\n",constraints," ",variables)
println("\n\n",c)
println("\n\n",A)



200 1000


[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,16.0,16.0,16.0,16.0,16.0,16.0,17.0,17.0,17.0,17.0,17.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,21.0,2

	[3   ,  115]  =  1.0
	[17  ,  115]  =  1.0
	[30  ,  115]  =  1.0
	[36  ,  115]  =  1.0
	[72  ,  115]  =  1.0
	[77  ,  115]  =  1.0
	[99  ,  115]  =  1.0
	[5   ,  116]  =  1.0
	[22  ,  116]  =  1.0
	[53  ,  116]  =  1.0
	[57  ,  116]  =  1.0
	[107 ,  116]  =  1.0
	[158 ,  116]  =  1.0
	[12  ,  117]  =  1.0
	[136 ,  117]  =  1.0
	[139 ,  117]  =  1.0
	[157 ,  117]  =  1.0
	[197 ,  117]  =  1.0
	[11  ,  118]  =  1.0
	[25  ,  118]  =  1.0
	[108 ,  118]  =  1.0
	[160 ,  118]  =  1.0
	[138 ,  119]  =  1.0
	[147 ,  119]  =  1.0
	[154 ,  119]  =  1.0
	[195 ,  119]  =  1.0
	[8   ,  120]  =  1.0
	[54  ,  120]  =  1.0
	[111 ,  120]  =  1.0
	[31  ,  121]  =  1.0
	[42  ,  121]  =  1.0
	[18  ,  122]  =  1.0
	[25  ,  122]  =  1.0
	[46  ,  122]  =  1.0
	[54  ,  122]  =  1.0
	[55  ,  122]  =  1.0
	[78  ,  122]  =  1.0
	[83  ,  122]  =  1.0
	[89  ,  122]  =  1.0
	[136 ,  122]  =  1.0
	[166 ,  122]  =  1.0
	[175 ,  122]  =  1.0
	[51  ,  123]  =  1.0
	[75  ,  123]  =  1.0
	[116 ,  123]  =  1.0
	[124 ,  1

	[17  ,  297]  =  1.0
	[49  ,  297]  =  1.0
	[71  ,  297]  =  1.0
	[109 ,  297]  =  1.0
	[113 ,  297]  =  1.0
	[17  ,  298]  =  1.0
	[124 ,  298]  =  1.0
	[147 ,  298]  =  1.0
	[2   ,  299]  =  1.0
	[50  ,  299]  =  1.0
	[192 ,  299]  =  1.0
	[75  ,  300]  =  1.0
	[142 ,  300]  =  1.0
	[38  ,  301]  =  1.0
	[103 ,  301]  =  1.0
	[28  ,  302]  =  1.0
	[32  ,  302]  =  1.0
	[77  ,  302]  =  1.0
	[120 ,  302]  =  1.0
	[136 ,  302]  =  1.0
	[20  ,  303]  =  1.0
	[106 ,  303]  =  1.0
	[129 ,  303]  =  1.0
	[138 ,  303]  =  1.0
	[104 ,  304]  =  1.0
	[160 ,  304]  =  1.0
	[185 ,  304]  =  1.0
	[23  ,  305]  =  1.0
	[86  ,  305]  =  1.0
	[101 ,  306]  =  1.0
	[104 ,  306]  =  1.0
	[9   ,  307]  =  1.0
	[28  ,  307]  =  1.0
	[41  ,  307]  =  1.0
	[71  ,  307]  =  1.0
	[155 ,  307]  =  1.0
	[163 ,  307]  =  1.0
	[67  ,  308]  =  1.0
	[72  ,  308]  =  1.0
	[138 ,  308]  =  1.0
	[152 ,  308]  =  1.0
	[198 ,  308]  =  1.0
	[108 ,  309]  =  1.0
	[154 ,  309]  =  1.0
	[165 ,  309]  =  1.0
	[172 ,  3

	[36  ,  447]  =  1.0
	[71  ,  447]  =  1.0
	[119 ,  447]  =  1.0
	[76  ,  448]  =  1.0
	[99  ,  448]  =  1.0
	[163 ,  448]  =  1.0
	[16  ,  449]  =  1.0
	[60  ,  449]  =  1.0
	[134 ,  449]  =  1.0
	[28  ,  450]  =  1.0
	[29  ,  450]  =  1.0
	[94  ,  450]  =  1.0
	[17  ,  451]  =  1.0
	[61  ,  451]  =  1.0
	[103 ,  451]  =  1.0
	[128 ,  451]  =  1.0
	[187 ,  451]  =  1.0
	[23  ,  452]  =  1.0
	[79  ,  452]  =  1.0
	[93  ,  452]  =  1.0
	[101 ,  452]  =  1.0
	[196 ,  452]  =  1.0
	[43  ,  453]  =  1.0
	[133 ,  453]  =  1.0
	[135 ,  453]  =  1.0
	[139 ,  453]  =  1.0
	[19  ,  454]  =  1.0
	[46  ,  454]  =  1.0
	[62  ,  454]  =  1.0
	[68  ,  454]  =  1.0
	[22  ,  455]  =  1.0
	[47  ,  455]  =  1.0
	[78  ,  455]  =  1.0
	[161 ,  455]  =  1.0
	[21  ,  456]  =  1.0
	[44  ,  456]  =  1.0
	[105 ,  456]  =  1.0
	[54  ,  457]  =  1.0
	[108 ,  457]  =  1.0
	[180 ,  457]  =  1.0
	[94  ,  458]  =  1.0
	[123 ,  458]  =  1.0
	[7   ,  459]  =  1.0
	[39  ,  459]  =  1.0
	[109 ,  459]  =  1.0
	[127 ,  4

	[190 ,  630]  =  1.0
	[23  ,  631]  =  1.0
	[86  ,  631]  =  1.0
	[156 ,  631]  =  1.0
	[171 ,  631]  =  1.0
	[191 ,  631]  =  1.0
	[9   ,  632]  =  1.0
	[44  ,  632]  =  1.0
	[93  ,  632]  =  1.0
	[184 ,  632]  =  1.0
	[45  ,  633]  =  1.0
	[61  ,  633]  =  1.0
	[127 ,  633]  =  1.0
	[151 ,  633]  =  1.0
	[11  ,  634]  =  1.0
	[100 ,  634]  =  1.0
	[116 ,  634]  =  1.0
	[138 ,  634]  =  1.0
	[3   ,  635]  =  1.0
	[10  ,  635]  =  1.0
	[107 ,  635]  =  1.0
	[153 ,  635]  =  1.0
	[3   ,  636]  =  1.0
	[24  ,  636]  =  1.0
	[30  ,  636]  =  1.0
	[152 ,  636]  =  1.0
	[40  ,  637]  =  1.0
	[125 ,  637]  =  1.0
	[154 ,  637]  =  1.0
	[10  ,  638]  =  1.0
	[14  ,  638]  =  1.0
	[109 ,  638]  =  1.0
	[64  ,  639]  =  1.0
	[142 ,  639]  =  1.0
	[2   ,  640]  =  1.0
	[7   ,  640]  =  1.0
	[35  ,  640]  =  1.0
	[106 ,  640]  =  1.0
	[109 ,  640]  =  1.0
	[124 ,  640]  =  1.0
	[143 ,  640]  =  1.0
	[190 ,  640]  =  1.0
	[24  ,  641]  =  1.0
	[56  ,  641]  =  1.0
	[70  ,  641]  =  1.0
	[113 ,  6

	[179 ,  784]  =  1.0
	[183 ,  784]  =  1.0
	[187 ,  784]  =  1.0
	[190 ,  784]  =  1.0
	[19  ,  785]  =  1.0
	[58  ,  785]  =  1.0
	[109 ,  785]  =  1.0
	[159 ,  785]  =  1.0
	[183 ,  785]  =  1.0
	[184 ,  785]  =  1.0
	[32  ,  786]  =  1.0
	[48  ,  786]  =  1.0
	[96  ,  786]  =  1.0
	[98  ,  786]  =  1.0
	[144 ,  786]  =  1.0
	[146 ,  786]  =  1.0
	[59  ,  787]  =  1.0
	[71  ,  787]  =  1.0
	[129 ,  787]  =  1.0
	[187 ,  787]  =  1.0
	[196 ,  787]  =  1.0
	[66  ,  788]  =  1.0
	[88  ,  788]  =  1.0
	[109 ,  788]  =  1.0
	[130 ,  788]  =  1.0
	[131 ,  788]  =  1.0
	[13  ,  789]  =  1.0
	[94  ,  789]  =  1.0
	[152 ,  789]  =  1.0
	[160 ,  789]  =  1.0
	[186 ,  789]  =  1.0
	[4   ,  790]  =  1.0
	[6   ,  790]  =  1.0
	[69  ,  790]  =  1.0
	[121 ,  790]  =  1.0
	[174 ,  790]  =  1.0
	[116 ,  791]  =  1.0
	[118 ,  791]  =  1.0
	[155 ,  791]  =  1.0
	[161 ,  791]  =  1.0
	[60  ,  792]  =  1.0
	[94  ,  792]  =  1.0
	[142 ,  792]  =  1.0
	[143 ,  792]  =  1.0
	[3   ,  793]  =  1.0
	[9   ,  7

	[99  ,  948]  =  1.0
	[110 ,  948]  =  1.0
	[199 ,  948]  =  1.0
	[9   ,  949]  =  1.0
	[45  ,  949]  =  1.0
	[49  ,  949]  =  1.0
	[174 ,  949]  =  1.0
	[64  ,  950]  =  1.0
	[99  ,  950]  =  1.0
	[126 ,  950]  =  1.0
	[135 ,  950]  =  1.0
	[27  ,  951]  =  1.0
	[65  ,  951]  =  1.0
	[87  ,  951]  =  1.0
	[142 ,  951]  =  1.0
	[36  ,  952]  =  1.0
	[40  ,  952]  =  1.0
	[149 ,  952]  =  1.0
	[168 ,  952]  =  1.0
	[138 ,  953]  =  1.0
	[142 ,  953]  =  1.0
	[184 ,  953]  =  1.0
	[7   ,  954]  =  1.0
	[35  ,  954]  =  1.0
	[126 ,  954]  =  1.0
	[64  ,  955]  =  1.0
	[107 ,  955]  =  1.0
	[186 ,  955]  =  1.0
	[24  ,  956]  =  1.0
	[39  ,  956]  =  1.0
	[178 ,  957]  =  1.0
	[200 ,  957]  =  1.0
	[38  ,  958]  =  1.0
	[48  ,  958]  =  1.0
	[59  ,  958]  =  1.0
	[60  ,  958]  =  1.0
	[68  ,  958]  =  1.0
	[76  ,  958]  =  1.0
	[171 ,  958]  =  1.0
	[181 ,  958]  =  1.0
	[35  ,  959]  =  1.0
	[105 ,  959]  =  1.0
	[106 ,  959]  =  1.0
	[117 ,  959]  =  1.0
	[134 ,  959]  =  1.0
	[144 ,  9